In [12]:
pip install pymysql

     |████████████████████████████████| 43 kB 3.0 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import os
import pymysql
from sqlalchemy import create_engine

In [2]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/CBARawFiles/')


In [3]:
with pd.ExcelFile('./Final Data NELLORE.xlsx') as xls:
    df1 = pd.read_excel(xls, 'Class 1 & 2')
    df2 = pd.read_excel(xls, 'Class 3 & 4 & 5')
    df3 = pd.read_excel(xls, 'Class 6 & 7 & 8')

In [4]:
df = pd.concat([df1,df2,df3], axis=0, ignore_index=True)

In [5]:
df.drop(df.loc[:, 'NAME':'Barcode'].columns, axis=1, inplace=True)

In [6]:
df.reset_index(drop=True, inplace=True)

In [7]:
# DEFINE THE DATABASE CREDENTIALS
user = 'square'
password = 'password'
host = '127.0.0.1'
port = 3306
database = 'cba'
  
# PYTHON FUNCTION TO CONNECT TO THE MYSQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )
  
  
if __name__ == '__main__':
  
    try:
        
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
        engine = get_connection()
        print(
            f"Connection to the {host} for user {user} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)

Connection to the 127.0.0.1 for user square created successfully.


In [47]:
df.to_sql('student_responses', con = engine, if_exists = 'append', chunksize = 1000, index= False)

165769

In [3]:
mapping = pd.read_csv('../MappingFiles/cba1_answer_key_with_question_info.csv')

In [21]:
mapping.to_sql('answer_key',con=engine,if_exists='append',chunksize = 1000,index=False)

1004

In [8]:
var_list = list(df)

In [9]:
dimensions = ['CHILDID']

In [10]:
retain = [ele for ele in var_list if ele not in dimensions]

In [11]:
df_unpivot = pd.melt(df,id_vars = dimensions
                        ,value_vars = retain)

In [60]:
df_unpivot.to_sql('student_responses_long', con = engine, if_exists = 'append', chunksize = 1000, index= False)

14919210

In [8]:
mapping_short = mapping[['project_name','subject_qno','correct_answer']]

In [12]:
mapping_short.head()

,project_name,subject_qno,correct_answer
0,SALT,E1,C
1,SALT,E2,B
2,SALT,E3,C
3,SALT,E4,B
4,SALT,E5,A


In [10]:
mapping_wide = pd.pivot(mapping_short,index='project_name',columns='subject_qno',values = 'correct_answer')

ValueError: Index contains duplicate entries, cannot reshape